In [1]:
import pandas as pd
import sqlite3

import sys
import os
# Modify path temporarily to import functions from NASEM_functions directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from NASEM_functions.ration_balancing_equations import *

In [8]:
# User will type the ration into input.txt
# This function reads that file and creates a dataframe with all of the feeds and % DM 

def read_input(input):
    data = []
    animal_input = {}

    with open(input, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith('#') or ':' not in line:
                continue

            if line.startswith('*'):
                key, value = line[1:].split(":")
                animal_input[key.strip()] = float(value.strip())
                continue

            feedstuff, per_DM = line.split(":")
            data.append([feedstuff.strip(), per_DM.strip()])

    user_input = pd.DataFrame(data, columns=["Feedstuff", "%_DM_user"])
    user_input['%_DM_user'] = user_input['%_DM_user'].astype(float)
    return user_input, animal_input

user_input, animal_input = read_input('input.txt')
list_of_feeds = user_input['Feedstuff'].tolist()

In [32]:
# Takes a list of the feed names and gets all their info from the feed library

def fl_get_rows(feeds_to_get):
    conn = sqlite3.connect('../../diet_database.db')
    cursor = conn.cursor()

    index_str = ', '.join([f"'{idx}'" for idx in feeds_to_get])

    query = f"SELECT * FROM feed_library WHERE Name IN ({index_str})"

    cursor.execute(query)
    rows = cursor.fetchall()

    cursor.execute(f"PRAGMA table_info(feed_library)")
    column_names = [column[1] for column in cursor.fetchall()]

    # Create a DataFrame from the retrieved rows with column names
    feed_data = pd.DataFrame(rows, columns=column_names)

    conn.close()

    return feed_data

feed_data = fl_get_rows(list_of_feeds)

In [15]:
# Get %DM to add to 100%
def set_perc_100():
    user_perc = user_input['%_DM_user'].sum()
    scaling_factor = 100 / user_perc

    user_input['%_DM_intake'] = user_input['%_DM_user'] * scaling_factor

    # Adjust so sum is exactly 100
    adjustment = 100 - user_input['%_DM_intake'].sum()
    user_input['%_DM_intake'] += adjustment / len(user_input)

    return user_input

user_input = set_perc_100()

In [9]:
# Predict DMI

def dmi_predicted(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp):
    DMI = calculate_Dt_DMIn_Lact1(An_Parity_rl, Trg_MilkProd, An_BW, An_BCS, An_LactDay, Trg_MilkFatp, Trg_MilkTPp, Trg_MilkLacp)
    return DMI
animal_input['DMI'] = dmi_predicted(**animal_input)


In [13]:
# Get kg intake of each feed

def get_kg_intake(df, dict):
    DMI = dict['DMI']
    df['kg_intake'] = df['%_DM_intake']/100 * DMI

get_kg_intake(user_input, animal_input)